In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import gc

gc.collect()

263

In [5]:
from huggingface_hub import login
import os

token_file_path = 'secrets.txt'

if os.path.exists(token_file_path):
    try:
        with open(token_file_path, 'r') as f:
            hf_token = f.read().strip()  # .strip() removes any leading/trailing whitespace

        login(token=hf_token)
        print("Successfully logged in to Hugging Face!")

    except Exception as e:
        print(f"An error occurred while trying to read the token file or log in: {e}")
else:
    print(f"Token file not found at {token_file_path}. Please create the file and add your token.")

/home/jovyan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully logged in to Hugging Face!


In [7]:
from src.data_utils import SIFT50MDataset
import os
from datasets import load_dataset, Dataset

sift_dataset = load_dataset(
    'amazon-agi/SIFT-50M',
    #name='closed_ended_comparison',
    name = 'closed_ended_content_level',
    split='train',
    trust_remote_code=True
)

sift_dataset_test = load_dataset(
    'amazon-agi/SIFT-50M',
    #name='closed_ended_comparison',
    name = 'closed_ended_content_level',
    split='EvalSIFT',
    trust_remote_code=True
) 

# Define the allowed data sources
allowed_values = ["common_voice_de", "common_voice_en", "vctk_en"]

# Define a filter function
def filter_data_source(example):
    return example["data_source"] in allowed_values

filtered_sift_data = sift_dataset.filter(filter_data_source)
filtered_sift_data_test = sift_dataset_test.filter(filter_data_source)

#df = sift_dataset.select(range(1000)).to_pandas()
print(len(filtered_sift_data))
print(len(filtered_sift_data_test))
#filtered_df = df[df["data_source"].isin(allowed_values)]

# Count number of entries per value
#counts = filtered_df["data_source"].value_counts()
#print(counts)
#sift_data = Dataset.from_pandas(filtered_df)
# Define the base datasets paths (replace with your actual paths)

'''print('Training set')
counts = filtered_sift_data.to_pandas()["data_source"].value_counts()
print(counts)
print('Evaluation set')
counts = filtered_sift_data_eval.to_pandas()["data_source"].value_counts()'''


Filter: 100%|██████████| 2500/2500 [00:00<00:00, 31631.73 examples/s]

3966476
601


'print(\'Training set\')\ncounts = filtered_sift_data.to_pandas()["data_source"].value_counts()\nprint(counts)\nprint(\'Evaluation set\')\ncounts = filtered_sift_data_eval.to_pandas()["data_source"].value_counts()'

In [5]:
print(counts)

data_source
common_voice_en    15293
common_voice_de    10261
Name: count, dtype: int64


In [6]:
del counts
gc.collect()

0

In [16]:
filtered_df.iloc[4].messages

array([{'role': 'user', 'content': array([{'text': None, 'audio_path': '/path/to/common_voice_de_27994683.wav'},
              {'text': 'Wie viele Endrundenteilnehmer wurden in jeder Gruppe eingeteilt?', 'audio_path': None}],
             dtype=object)}                                                                                     ,
       {'role': 'assistant', 'content': array([{'text': 'Vier Mannschaften wurden in jeder Gruppe eingeteilt.', 'audio_path': None}],
             dtype=object)}                                                                                                          ],
      dtype=object)

In [1]:
import pandas as pd

csv_de = pd.read_csv('./data/common_voice_de_eval_mapping.csv')
filtered_mask=csv_de['audio_path'].str.contains('common_voice_de_17733078')

print(csv_de[filtered_mask])

Empty DataFrame
Columns: [Unnamed: 0, id, audio_path]
Index: []


In [15]:
dataset = load_dataset("mozilla-foundation/common_voice_15_0", 'en', split="train", trust_remote_code=True)
print(len(dataset))

1070066


In [ ]:
filtered_df_1 = filtered_df[filtered_df["data_source"].isin(['vctk_en'])]
print(filtered_df_1.iloc[1]['messages']) # filterining for a specific dataset

In [4]:
total_len = len(sift_data)
eval_len = int(0.2*total_len)

eval_ds = sift_data.select(range(eval_len))
train_ds = sift_data.select(range(eval_len,total_len))


In [8]:
base_datasets_root = "/home/jovyan/.cache/huggingface/datasets"
base_datasets_paths = {
    "common_voice_de": None, # No longer needs a path, handled by load_dataset
    #"multilingual_librispeech_de": None, # No longer needs a path, handled by load_dataset
    "common_voice_en": None, # No longer needs a path, handled by load_dataset
    "vctk_en": "./vctk_corpus" # VCTK still needs a root path for torchaudio
}

In [10]:
sift_iterable_dataset_eval = SIFT50MDataset(sift_dataset=filtered_sift_data_test, base_datasets_paths=base_datasets_paths, test_set=True)

for x in sift_iterable_dataset_eval:
    print(x)
    break

100%|██████████| 16167/16167 [00:06<00:00, 2542.99it/s]


Common Voice de mapping saved to ./data/common_voice_de_test_mapping.csv


100%|██████████| 16386/16386 [00:06<00:00, 2547.45it/s]


Common Voice en mapping saved to ./data/common_voice_en_test_mapping.csv
['common_voice_de_25543288']
common_voice_de_25543288
/home/jovyan/.cache/huggingface/datasets/downloads/extracted/e2d0a7ef8aa3a7692bd12936cfc8b5ca687d58a4fedbd18ac6820dbe1aef6b8d/de_test_0/common_voice_de_25543288.mp3
{'id': 'common_voice_de_25543288', 'messages': [{'role': 'user', 'content': [{'text': None, 'audio_path': '/home/jovyan/.cache/huggingface/datasets/downloads/extracted/e2d0a7ef8aa3a7692bd12936cfc8b5ca687d58a4fedbd18ac6820dbe1aef6b8d/de_test_0/common_voice_de_25543288.mp3', 'type': 'audio'}, {'text': 'Ist das Audio für ein erwachsenes Publikum geeignet?', 'audio_path': None, 'type': 'text'}]}, {'role': 'assistant', 'content': [{'text': 'Ja, das Audio ist für ein erwachsenes Publikum geeignet.', 'audio_path': None}]}], 'task': 'closed_ended_content_level', 'data_source': 'common_voice_de'}
{'id': 'common_voice_de_25543288', 'messages': [{'role': 'user', 'content': [{'text': None, 'audio_path': '/home/

In [9]:
%reload_ext autoreload
    
from src.train_qlora import train_model
from transformers import AutoProcessor
from src.data_utils import SIFT50MDataset
from src.data_collator import CustomDataCollator


def main():
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
    data_collator = CustomDataCollator(processor)

    sift_iterable_dataset_eval = SIFT50MDataset(sift_dataset=filtered_sift_data_eval, base_datasets_paths=base_datasets_paths)
    sift_iterable_dataset_train = SIFT50MDataset(sift_dataset=filtered_sift_data, base_datasets_paths=base_datasets_paths)

    train_model( eval_ds = sift_iterable_dataset_eval,train_ds = sift_iterable_dataset_train, processor=processor, custom_data_collator=data_collator)

In [10]:
main()

Loading checkpoint shards: 100%|██████████| 5/5 [01:06<00:00, 13.27s/it]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151645, 'pad_token_id': 151643}.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
/home/jovyan/.local/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant paramete

Epoch,Training Loss,Validation Loss


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that migh

File not found: /path/to/common_voice_de_25313610.wav. Skipping this file.
No valid audio files found for a sample. Skipping.


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that migh

...

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


File not found: /path/to/common_voice_de_19173359.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_19968105.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_18432079.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_19956721.wav. Skipping this file.
No valid audio files found for a sample. Skipping.


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


File not found: /path/to/common_voice_de_20788669.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_20371012.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_21852940.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_18211172.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_20178337.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_21852939.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_19726775.wav. Skipping this file.
No valid audio files found for a sample. Skipping.
File not found: /path/to/common_voice_de_23416143.wav. Skipping this file.
No valid audio files found for a sample. Sk

IndexError: list index out of range